# Logistic Regression Consulting Classification

## Binary Customer Churn

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. 

The data fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('consult').getOrCreate()

In [3]:
data = spark.read.csv('customer_churn.csv',inferSchema=True, header=True)

In [4]:
data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



### Check out the data

In [5]:
data.describe().show()

+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|summary|        Names|              Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|            Location|             Company|              Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|  count|          900|              900|              900|               900|              900|               900|                 900|                 900|                900|
|   mean|         null|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777|                null|                null|0.16666666666666666|
| stddev|         null|6.127560416916251|2408.644531858096|0.4999208935073339|1.274449013194616|1.764835592035

In [6]:
data.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

### Format for MLlib

We'll ues the numerical columns. We'll include Account Manager.

In [8]:
from pyspark.ml.feature import VectorAssembler

In [9]:
assembler = VectorAssembler(inputCols=['Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites'],outputCol='features')

In [10]:
output = assembler.transform(data)

In [11]:
final_data = output.select('features','churn')

### Test Train Split

In [12]:
train_churn, test_churn = final_data.randomSplit([0.7, 0.3])

### Fit the model

In [13]:
from pyspark.ml.classification import LogisticRegression

In [14]:
lr_churn = LogisticRegression(labelCol='churn')

In [15]:
fitted_churn_model = lr_churn.fit(train_churn)

In [16]:
training_sum = fitted_churn_model.summary

In [17]:
training_sum.predictions.describe().show()

+-------+------------------+-------------------+
|summary|             churn|         prediction|
+-------+------------------+-------------------+
|  count|               636|                636|
|   mean|0.1650943396226415|0.12421383647798742|
| stddev|0.3715578914201275|0.33008494950086786|
|    min|               0.0|                0.0|
|    max|               1.0|                1.0|
+-------+------------------+-------------------+



### Evaluate results

Let's evaluate the results on the data set we were given (using the test data)

In [18]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [19]:
pred_and_labels = fitted_churn_model.evaluate(test_churn)

In [20]:
pred_and_labels.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[22.0,11254.38,1....|    0|[4.34906620256974...|[0.98724589803449...|       0.0|
|[26.0,8939.61,0.0...|    0|[6.18838881202767...|[0.99795107464284...|       0.0|
|[28.0,8670.98,0.0...|    0|[7.61606826208486...|[0.99950776810202...|       0.0|
|[28.0,9090.43,1.0...|    0|[1.30873268507574...|[0.78730101096180...|       0.0|
|[28.0,11128.95,1....|    0|[3.99608492422813...|[0.98194450855072...|       0.0|
|[28.0,11204.23,0....|    0|[1.62247903636751...|[0.83513673552370...|       0.0|
|[29.0,11274.46,1....|    0|[4.31433032529988...|[0.98680103902761...|       0.0|
|[29.0,13255.05,1....|    0|[3.99713022936459...|[0.98196303194501...|       0.0|
|[30.0,10183.98,1....|    0|[2.71408397264037...|[0.93785260942174...|       0.0|
|[30.0,11575.37,

### Using AUC

In [21]:
churn_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                           labelCol='churn')

In [22]:
auc = churn_eval.evaluate(pred_and_labels.predictions)

In [23]:
auc

0.7817351598173516

### Predict on brand new unlabeled data

new_customers.csv file!

In [24]:
# Fit new model on all the complete data (train + test)
final_lr_model = lr_churn.fit(final_data)

In [25]:
new_customers = spark.read.csv('new_customers.csv',inferSchema=True,
                              header=True)

In [26]:
new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)



In [27]:
test_new_customers = assembler.transform(new_customers)

In [28]:
test_new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- features: vector (nullable = true)



In [29]:
final_results = final_lr_model.transform(test_new_customers)

In [30]:
final_results.select('Company','prediction').show()

+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+



Ok! That is it! Now we know that we should assign Acocunt Managers to Cannon-Benson, Barron-Robertson, Sexton-GOlden, and Parks-Robbins!

## Great Job!